![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [635]:
import pandas as pd
import numpy as np

# Start coding here...
data=pd.read_csv("bank_marketing.csv")
data.head()


,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Client table

In [636]:
client =data[["client_id","age","job","marital","education","credit_default","housing","loan"]]
client.rename(columns={"client_id":"id"},inplace=True)


In [637]:
client['education'] = client['education'].replace('.', '_').replace('unknown', np.nan)
client['job'] = client['job'].str.replace('.', '')

In [638]:
#change from object  to boolean
client['housing'] = client['housing'].replace({'yes': True, 'no': False}).astype("bool")
client['loan'] = client['loan'].replace({'yes': True, 'no': False}).astype("bool")
client['credit_default'] = client['credit_default'].replace({'yes': True, 'no': False}).astype("bool")

In [639]:
#client.head()


In [640]:
#client.info()

## Economics table

In [641]:
economics =data[["client_id","emp_var_rate","cons_price_idx","euribor3m","nr_employed"]]
economics.rename(columns={"euribor3m":"euribor_three_months","nr_employed":"number_employed"},inplace=True)


In [642]:
#economics.head()

In [643]:
#economics.info()

## Campaign table

In [644]:
campaign = data[["campaign", "client_id","duration","pdays", "previous","poutcome","month","day","y"]]
campaign.rename(columns={"poutcome": "previous_outcome","duration":"contact_duration","campaign":"number_contacts","previous":"previous_campaign_contacts","y":"campaign_outcome"}, inplace=True)


In [645]:

#change from object  to boolean
campaign['previous_outcome']  =campaign['previous_outcome'] .replace({'success': True, 'failure': False}).astype("bool")
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'success': True, 'failure': False}).astype("bool")
# Change "nonexistent" to NumPy's null values in "previous_outcome" column
campaign['previous_outcome'] =  campaign['previous_outcome'].replace('nonexistent', np.nan)


In [646]:
# Map lowercase month abbreviations to month numbers
month_mapping = {
    'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'
}

# Create the last_contact_date column
campaign['last_contact_date'] = pd.to_datetime('2022-' + campaign['month'].map(month_mapping) + '-' + campaign['day'].astype(str))


In [647]:
campaign['campaign_id'] = 1
campaign = campaign[["campaign_id",
    "client_id",
    "number_contacts",
    "contact_duration",
    "pdays",
    "previous_campaign_contacts",
    "previous_outcome",
    "campaign_outcome",
    "last_contact_date"
    ]]

In [648]:
campaign = campaign.drop(['day', 'month'], axis=1, errors='ignore')

In [649]:
#campaign.head()



In [650]:
#campaign.info()

## Saving DataFrames

In [651]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)


## SQL

In [652]:

# SQL code for creating the client table
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;
"""

# SQL code for creating the campaign table
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;
"""

# SQL code for creating the economics table
economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;
"""
